In [45]:
# Importar Librerias
from google.colab import files
import math
import numpy as np
from scipy import ndimage

In [46]:
# Subir Archivos
files.upload()  

Saving Data.txt to Data (2).txt


{'Data.txt': b'7654598954321095410125798754578999894323456789349878901298743234767897899987654234567895493239656798\r\n6543487895992987324234599543467998789934567893298767892987643123457976789998762123456789989198768987\r\n7632356789889876535345987654788987678897678954997656789998784434578965699999878244678997978999999876\r\n7543487896673987676767898995689876556789789769876545698999896549989434988799954345989896767899886765\r\n9854567965432398989898959987789987347899899879765434567893987698999549877669875656798765656998765454\r\n8767778954321239799979545299897898258976989989898545679932198987987698765457989767897654343987654323\r\n9878989765932345689965432134956789349965678999987656998991019876798987654346799898998743212398787634\r\n9989999979896456789876541012347898767894567899898769897989198765689996543235678969987632101239896545\r\n9999987898789767899985432123456789898993678998789898796778987654677897652124689659876543212548987656\r\n8999876765678998989996543234567899969789799987678

# Parte 1

In [61]:
# Con el archivo abierto
with open("Data.txt") as f:
    # Transformamos los datos en una matriz con las alturas
    heights = np.genfromtxt(f, delimiter=1, dtype=int)

    # Agregamos un padding, espacio con valor de 10 en los bordes para la matriz
    matrix = np.pad(alturas, pad_width=1, mode="constant", constant_values=10)

    # Implementamos la función roll para recorrer los valores en el eje horizontal y vertical
    # El desplazamiento será de 1 espacio o de -1 espacio
    minimum = ((matrix < np.roll(matrix,  1, axis=0)) & (matrix < np.roll(matrix, -1, axis=0))
            & (matrix < np.roll(matrix,  1, axis=1)) & (matrix < np.roll(matrix, -1, axis=1)))

    # Resultados
    # Por lo tanto el resultado será la suma de los valores mínimos + 1
    print('La suma es: ' + str(np.sum(matrix[minimum] + 1)))

La suma es: 512


# Parte 2

In [63]:
# Empleamos herramientas de la librería scipy
# Esta nos brindará herramientas para solucionar
# fácilmente este problema

# Con el archivo abierto
with open("Data.txt") as f:
    # Transformamos los datos en una matriz con las alturas en formato entero
    matrix = np.array([[*map(int, line)] for line in f.read().splitlines()])

    # Definimos un arreglo con los valores adyacentes
    near = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]])
    # Implementamos ndimage para tratar la matriz como si fuese una imagen para procesarla multidimensionalmente
    # Usamos como parámetros la matriz de datos, el tamaño 1 y el tamaño de footprint se define
    # entre los valores adyacentes, determinando que la extensión de la matriz más
    # allá de sus límites como constante y los valores con los que se llenará los bordes
    deepest_points = ndimage.rank_filter(matrix, 1, footprint=near, mode="constant", cval=9.0)
    # Definimos un borde de la cuenca 
    deepest_border = matrix < deepest_points
    # Aplicamos una función morgológica de dilatación binaria, considerando el borde de la cuenca
    # consideramos a los elementos cercanos (near) para estructurar esta dilatación,
    # se recorre 1 lugar, su mask filtra valores para que solo pase lo necesario en cada iteración
    basins_mask = ndimage.binary_dilation(deepest_border, structure=near, iterations=-1, mask=dem != 9)
    # Procedemos a etiquetar las cuencas, con las mascarás de las cuencas como entrada y
    # teniendo en cuenta los valores adyacentes como estructura de conexiones
    basins, basins_n = ndimage.label(basins_mask, structure=near)
    # Obtenemos el numero de cuencas y el tamaño de las mismas
    n_basins, basin_size = np.unique(basins[basins>0], return_counts=True)

    # Resultados
    # Por lo tanto el resultado será la multiplicación de las 3 últimas cuencas
    # que ordenadas por tamaño son las 3 más grandes
    print("La multiplicación de las 3 cuencas más grandes: "  + str(np.product(basin_sizes[np.argsort(basin_sizes)[-3:]])))

La multiplicación de las 3 cuencas más grandes: 1600104
